# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_library 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_library')
except Exception as e:
    print(e)

## Query 1

Give me:

- the artist
- song title 
- song's length in the music app history that was heard during
for sessionId = 338, and itemInSession = 4


In [8]:
table1 = "artist_songs_in_session"
select1 = "select artist, song, length from " + table1 + " WHERE sessionId = 338 and itemInSession = 4"

drop1 = "DROP TABLE IF EXISTS " + table1 + " "
try:
    rows = session.execute(drop1)
except Exception as e:
    print(e)
create1 = "CREATE TABLE IF NOT EXISTS " + table1 + " "
create1 = create1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create1)
except Exception as e:
    print(e)
    
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO " + table1 + " (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verify that the data have been inserted into each table

In [10]:
try:
    rows = session.execute(select1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2

Provides:
- name of artist, 
- song (sorted by itemInSession) 
- user (first and last name)
for userid = 10, sessionid = 182

In [11]:
table2 = "artist_songs_for_user_session"
select2 = "select artist, song, firstName, lastName from " + table2 + " WHERE userId = 10 and sessionId = 182"

drop2 = "DROP TABLE IF EXISTS " + table2 + " "
try:
    rows = session.execute(drop2)
except Exception as e:
    print(e)
    
create2 = "CREATE TABLE IF NOT EXISTS " + table2 + " "
create2 = create2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(create2)
except Exception as e:
    print(e)                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO " + table2 + " (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [13]:
try:
    rows = session.execute(select2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3

Provides:
- user name (first and last) in my music app history 
- who listened to the song 'All Hands Against His Own'


In [14]:
table3 = "users_for_song"
select3 = "select firstName, lastName from " + table3 + " WHERE song = 'All Hands Against His Own'"

drop3 = "DROP TABLE IF EXISTS " + table3 + " "
try:
    rows = session.execute(drop3)
except Exception as e:
    print(e)
    
create3 = "CREATE TABLE IF NOT EXISTS " + table3 + " "
create3 = create3 + "(song text, firstName text, lastName text, PRIMARY KEY ((song), firstName, lastName))"
try:
    session.execute(create3)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO " + table3 + " (song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))


In [16]:
try:
    rows = session.execute(select3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Cleanup all the tables before closing out the sessions

In [17]:
try:
    session.execute(drop1)
    session.execute(drop2)
    session.execute(drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()